In [1]:
import requests
import pandas as pd
from functools import *
import numpy as np
import nltk
%load_ext autoreload
%autoreload 2
import recip as rr
import dill

In [2]:
inp = ["flour","butter","milk", "egg", "cheese"]

In [3]:
# allr = rr.GetRecipies(inp)
# allr.get()

In [4]:
allr = rr.GetRecipies(inp)
dbfile = open('resp.pkl', 'rb') 
dbfile.seek(0)
allr.response =  dill.load(dbfile)
allr.process()

In [5]:
r = allr.recser.iloc[0]

In [6]:
dbrfile = open('respr.pkl', 'rb') 
r.response =  dill.load(dbrfile)
r.setVars()

In [7]:
print(r.missedIngs)

[<recip.Ingredient object at 0x13572a750>, <recip.Ingredient object at 0x13572a550>, <recip.Ingredient object at 0x13572a690>, <recip.Ingredient object at 0x13572a790>]


In [8]:
from difflib import SequenceMatcher
from bs4 import BeautifulSoup

In [9]:

req  = requests.get("https://www.allrecipes.com/article/common-ingredient-substitutions/")
data = req.text
soup = BeautifulSoup(data)
t = soup.findAll("div", {"class":"component table"})

In [10]:
nt = soup.findAll('table')[0]

In [11]:
rows = nt.findAll('tr')
rowtext = lambda cols: [cols[i].get_text().strip("\n") for i in [0,2]] #range(len(cols))
tabletext = [rowtext(rows[i].findAll('td')) for i in range(1,len(rows))]
tbdf = pd.DataFrame(tabletext)
tbdf.columns = ["from","tou"]

In [12]:
colsmax = max(tbdf['tou'].map(lambda x: x.count("OR")))

In [13]:
tbdf['tof'] = tbdf['tou'].map(lambda x: x.split("OR"))

In [14]:
pd.set_option('max_colwidth', 2000)
tbdf

,from,tou,tof
0,Allspice,"1/2 teaspoon cinnamon, 1/4 teaspoon ginger, and 1/4 teaspoon cloves","[1/2 teaspoon cinnamon, 1/4 teaspoon ginger, and 1/4 teaspoon cloves]"
1,Arrowroot starch,1 tablespoon flour OR 1 teaspoon cornstarch,"[1 tablespoon flour , 1 teaspoon cornstarch]"
2,Baking mix,1 cup pancake mix OR 1 cup Easy Biscuit Mixture,"[1 cup pancake mix , 1 cup Easy Biscuit Mixture]"
3,Baking powder,1/4 teaspoon baking soda plus 1/2 teaspoon cream of tartar OR 1/4 teaspoon baking soda plus 1/2 cup buttermilk (decrease liquid in recipe by 1/2 cup),"[1/4 teaspoon baking soda plus 1/2 teaspoon cream of tartar , 1/4 teaspoon baking soda plus 1/2 cup buttermilk (decrease liquid in recipe by 1/2 cup)]"
4,Baking soda,"4 teaspoons baking powder OR 1 teaspoon potassium bicarbonate and 1/3 teaspoon salt. NOTE: If the recipe calls for an acidic liquid such as sour cream, yogurt, buttermilk, vinegar, molasses, or citrus juice, you should replace it with the same amount of whole milk","[4 teaspoons baking powder , 1 teaspoon potassium bicarbonate and 1/3 teaspoon salt. NOTE: If the recipe calls for an acidic liquid such as sour cream, yogurt, buttermilk, vinegar, molasses, or citrus juice, you should replace it with the same amount of whole milk]"
5,Beer,1 cup nonalcoholic beer OR 1 cup chicken broth,"[1 cup nonalcoholic beer , 1 cup chicken broth]"
6,Brandy,1 teaspoon imitation brandy extract plus enough water to make 1/4 cup,[1 teaspoon imitation brandy extract plus enough water to make 1/4 cup]
7,Bread crumbs,1 cup cracker crumbs OR 1 cup matzo meal OR 1 cup ground oats,"[1 cup cracker crumbs , 1 cup matzo meal , 1 cup ground oats]"
8,Broth: beef or chicken,1 bouillon cube plus 1 cup boiling water OR 1 tablespoon soy sauce plus enough water to make 1 cup OR 1 cup vegetable broth,"[1 bouillon cube plus 1 cup boiling water , 1 tablespoon soy sauce plus enough water to make 1 cup , 1 cup vegetable broth]"
9,Brown sugar,1 cup white sugar plus 1/4 cup molasses and decrease the liquid in recipe by 1/4 cup OR 1 cup white sugar OR 1 1/4 cups confectioners' sugar,"[1 cup white sugar plus 1/4 cup molasses and decrease the liquid in recipe by 1/4 cup , 1 cup white sugar , 1 1/4 cups confectioners' sugar]"


In [32]:
def strdist(x,y):
    if x in y:
        return True
    return True if SequenceMatcher(None, x, y).ratio() > 0.5 else False

In [33]:
tbdf['from'] = tbdf['from'].map(lambda x: x.lower().replace("-"," "))

In [34]:
mi = [i.sname for i in r.missedIngs]

In [35]:
def matching(ingname):
    x = ingname.lower()
    tbdf['test'] = tbdf['from'].map(lambda y: strdist(x,y))
    return tbdf.sort_values('test',ascending = False)[tbdf.test == True][['from','tof']]

In [37]:
mi[3]

'unsalted butter'

In [38]:
matching(mi[3])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,from,tof
